In [15]:
import tkinter as tk 
from tkinter import *
from tkinter import filedialog
from PIL import Image
from PIL import ImageTk
import cv2
import numpy as np
import imutils
# importamos las librerias
from imutils.object_detection import non_max_suppression
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from PIL import Image as imagePIL
import numpy as np
import cv2
import imutils
import lbp
import matplotlib.pyplot as plt


In [16]:
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def deteccion_facilal(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceClassif.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # path del modelo entrenado
    modelo_path = 'modelo_multiclase.h5'
    video_path =path_video
    # path clases
    classes_path = 'clases_multiclases.txt'
    # cargamos las clases guardadas en el archivo classes.txt
    classes = np.loadtxt(classes_path, dtype=np.str)
    # instanciamos la fuente para el texto que se mostrara al identificar a una persona
    font = cv2.FONT_HERSHEY_SIMPLEX
    # traemos el video que analizaremos
    ########video = cv2.VideoCapture(video_path)#####cap
    # cargamos el modelo pre - entrenado
    our_model = keras.models.load_model(modelo_path)
    scale = 1.0
    # cargamos el modelo HOGDescriptor
    hog = cv2.HOGDescriptor()
    # cargamos los pesos para detectar personas en una imagen
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    # contador de frames
    count = 0
    aux = frame.copy()
    # resultados de la deteccion de personas en el frame
    (rects, weights) = hog.detectMultiScale(frame, winStride=(2, 2), padding=(4, 4), scale=1.05)
    idx = 0
    
    # recorrido del resultado de rects  para dibujar las cajas alrededor de las personas detectas
    for (x, y, w, h) in rects:
        # presicion de la deteccion
        confidence = float((weights[idx]))
# se tomara en cuenta la deteccion de una persona en el frame siempre que la exactitud sea mayor a 2.1
        if confidence >= 2.1:
            # captura de la seccion del frame, donde se ha detectado una persona
            body = aux[y:y + h, x:x + w, :]
            # transformamos la seccion de la imagen a PIL image
            # body = body.resize((100, 100))
            # redimensionamos la seccion que contiene a la persona, para poder pasarla a nuestra red
            body = image.array_to_img(x=body).resize(size=(100, 100))
            plt.imshow(body)
            plt.show()
            #body = cv2.resize(body, (100, 100))
            # tranformamos la imagen a un arreglo
            X = image.img_to_array(body)
            # expandimos las dimensiones del array
            X = np.expand_dims(X, axis=0)
            # guardamos esto en un numpy array
            images = np.vstack([X])
            # hacemos la prediccion para la persona detectada en la imagen
            val = our_model.predict(images)
            print(val)
            _, j = np.where(val == 1)
            label = classes[j]
            # dibujamos un rectangulo alrededor del peaton detectado
            cv2.rectangle(aux, (x, y), (x + w, y + h), (0, 0, 255), 2)
            # escribimos el nombre de la etiqueta
            cv2.putText(aux, "{}".format(label), (x, y + 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            idx += 1
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    # se dibujan los rectangulos finales
    for (xA, yA, xB, yB) in pick:
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
    aux = cv2.cvtColor(aux, cv2.COLOR_BGR2RGB)
return frame

In [17]:
def video_de_entrada():
    global cap, path_video
    if selected.get() == 1:
        path_video = filedialog.askopenfilename(filetypes = [
            ("all video format", ".mp4"),
            ("all video format", ".avi")])
        if len(path_video) > 0:
            btnEnd.configure(state="active")
            rad1Ele.configure(state="disabled")
            rad2Direc.configure(state="disabled")

            pathInputVideo = "..." + path_video[-20:]
            lblInfoVideoPath.configure(text=pathInputVideo)
            cap = cv2.VideoCapture(path_video)
            visualizar()
    if selected.get() == 2:
        btnEnd.configure(state="active")
        rad1Ele.configure(state="disabled")
        rad2Direc.configure(state="disabled")
        lblInfoVideoPath.configure(text="")
        cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        visualizar()

In [18]:
def visualizar():
    global cap
    ret, frame = cap.read()
    if ret == True:
        frame = imutils.resize(frame, width=640)
        frame = deteccion_facilal(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(frame)
        img = ImageTk.PhotoImage(image=im)

        lblVideo.configure(image=img)
        lblVideo.image = img
        lblVideo.after(10, visualizar)
    else:
        lblVideo.image = ""
        lblInfoVideoPath.configure(text="")
        rad1Ele.configure(state="active")
        rad2Direc.configure(state="active")
        selected.set(0)
        btnEnd.configure(state="disabled")
        cap.release()
        

In [19]:
def finalizar_limpiar():
    lblVideo.image = ""
    lblInfoVideoPath.configure(text="")
    rad1Ele.configure(state="active")
    rad2Direc.configure(state="active")
    selected.set(0)
    cap.release()

In [20]:
def elegir_imagen():
    # Especificar los tipos de archivos, para elegir solo a las imágenes
    path_image = filedialog.askopenfilename(filetypes = [
        ("image", ".jpeg"),
        ("image", ".png"),
        ("image", ".jpg")])

    if len(path_image) > 0:
        global image

        # Leer la imagen de entrada y la redimensionamos
        image = cv2.imread(path_image)
        image= imutils.resize(image, height=380)

        # Para visualizar la imagen de entrada en la GUI
        imageToShow= imutils.resize(image, width=180)
        imageToShow = cv2.cvtColor(imageToShow, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(imageToShow )
        img = ImageTk.PhotoImage(image=im)
    
        lblInputImage.configure(image=img)
        lblInputImage.image = img
        

        # Al momento que leemos la imagen de entrada, vaciamos
        # la iamgen de salida y se limpia la selección de los
        # radiobutton
        lblOutputImage.image = ""
        selected.set(0)

In [21]:
def deteccion_color():
    global image
    if selected.get() == 1:
        # Rojo
        rangoBajo1 = np.array([0, 140, 90], np.uint8)
        rangoAlto1 = np.array([8, 255, 255], np.uint8)
        rangoBajo2 = np.array([160, 140, 90], np.uint8)
        rangoAlto2 = np.array([180, 255, 255], np.uint8)

    if selected.get() == 2:
        # Amarillo
        rangoBajo = np.array([10, 98, 0], np.uint8)
        rangoAlto = np.array([25, 255, 255], np.uint8)

    if selected.get() == 3:
        # Azul celeste
        rangoBajo = np.array([88, 104, 121], np.uint8)
        rangoAlto = np.array([99, 255, 243], np.uint8)
        
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageGray = cv2.cvtColor(imageGray, cv2.COLOR_GRAY2BGR)
    imageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    if selected.get() == 1:
        # Detectamos el color rojo
        maskRojo1 = cv2.inRange(imageHSV, rangoBajo1, rangoAlto1)
        maskRojo2 = cv2.inRange(imageHSV, rangoBajo2, rangoAlto2)
        mask = cv2.add(maskRojo1, maskRojo2)
    else:
        # Detección para el color Amarillo y Azul celeste
        mask = cv2.inRange(imageHSV, rangoBajo, rangoAlto)

    mask = cv2.medianBlur(mask, 7)
    colorDetected = cv2.bitwise_and(image, image, mask=mask)

    # Fondo en grises
    invMask = cv2.bitwise_not(mask)
    bgGray = cv2.bitwise_and(imageGray, imageGray, mask=invMask)

    # Sumamos bgGray y colorDetected
    finalImage = cv2.add(bgGray, colorDetected)
    imageToShowOutput = cv2.cvtColor(finalImage, cv2.COLOR_BGR2RGB)

    # Para visualizar la imagen en lblOutputImage en la GUI
    im = Image.fromarray(imageToShowOutput)
    img = ImageTk.PhotoImage(image=im)
    lblOutputImage.configure(image=img)
    lblOutputImage.image = img
    # Label IMAGEN DE SALIDA
    lblInfo3 = Label(win, text="IMAGEN DE SALIDA:", font="bold")
    lblInfo3.grid(column=1, row=0, padx=5, pady=5)



In [22]:
def ventana2():
    global lblInputImage, lblOutputImage,selected,win

    win = tk.Toplevel()
    # Label donde se presentará la imagen de entrada
    lblInputImage = tk.Label(win)
    lblInputImage.grid(column=0, row=2)
    # Label donde se presentará la imagen de salida
    lblOutputImage = Label(win)
    lblOutputImage.grid(column=1, row=1, rowspan=6)

    lblInfo2 = Label(win, text="Analizar", width=25)
    lblInfo2.grid(column=0, row=3, padx=5, pady=5)
    # Label IMAGEN DE ENTRADA
    lblInfo1 = tk.Label(win, text="IMAGEN DE ENTRADA:")
    lblInfo1.grid(column=0, row=1, padx=5, pady=5)
    # Creamos los radio buttons y la ubicación que estos ocuparán
    selected = IntVar()
    rad1 = Radiobutton(win, text='Rojo', width=25,value=1, variable=selected, command= deteccion_color)
    rad2 = Radiobutton(win, text='Amarillo',width=25, value=2, variable=selected, command= deteccion_color)
    rad3 = Radiobutton(win, text='Azul celeste',width=25, value=3, variable=selected, command= deteccion_color)
    rad1.grid(column=0, row=4)
    rad2.grid(column=0, row=5)
    rad3.grid(column=0, row=6)
    # Creamos el botón para elegir la imagen de entrada
    btn = Button(win, text="Elegir imagen", width=25, command=elegir_imagen)
    btn.grid(column=0, row=0, padx=5, pady=5)


In [23]:
def ventanaVideo():
    global winVideo,lblInfo1Video,btnEleVideo,rad1Ele,rad2Direc,lblInfoVideoPath,lblVideo,btnEnd,selected,lblCapturaImegen

    cap = None
    winVideo = tk.Toplevel()
    ###################################
    lblCapturaImegen = Label(winVideo)
    lblCapturaImegen.grid(column=100, row=30, columnspan=2)
    ###################################


    lblInfo1Video = Label(winVideo, text="VIDEO DE ENTRADA", font="bold")
    lblInfo1Video.grid(column=0, row=0, columnspan=2)
    selected = IntVar()
    rad1Ele = Radiobutton(winVideo, text="Elegir video", width=20, value=1, variable=selected, command=video_de_entrada)
    rad2Direc = Radiobutton(winVideo, text="Video en directo", width=20, value=2, variable=selected, command=video_de_entrada)
    rad1Ele.grid(column=0, row=1)
    rad2Direc.grid(column=1, row=1)
    lblInfoVideoPath = Label(winVideo, text="", width=20)
    lblInfoVideoPath.grid(column=0, row=2)

    lblVideo = Label(winVideo)
    lblVideo.grid(column=0, row=3, columnspan=2)

    btnEnd = Button(winVideo, text="Finalizar visualización y limpiar", state="disabled", command=finalizar_limpiar)
    btnEnd.grid(column=0, row=4, columnspan=2, pady=10)




In [24]:


ventana= tk.Tk()

ventana.title("Sistema")

ventana.geometry('500x500')

lbl = Label(ventana, text="Sistema para el reconocimiento de personas")
lbl.place(x=70, y=10)

lblA = Label(ventana, text="Analizar")
lblA.place(x=10, y=70)

btnV =Button(ventana, text="Video", command = ventanaVideo)
btnV.place(x=90, y=50, width=100, height=30)

btn = Button(ventana, text="Imagen", command = ventana2)
btn.place(x=90, y=90, width=100, height=30)

ventana.mainloop()

<ipython-input-16-97edc8b75609>:13: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  classes = np.loadtxt(classes_path, dtype=np.str)
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\miniconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-17-85cb227bf119>", line 15, in video_de_entrada
    visualizar()
  File "<ipython-input-18-986475f894f5>", line 6, in visualizar
    frame = deteccion_facilal(frame)
  File "<ipython-input-16-97edc8b75609>", line 30, in deteccion_facilal
    (rects, weights) = hog.detectMultiScale(frame, winStride=(2, 2), padding=(4, 4), scale=1.05)
KeyboardInterru

KeyboardInterrupt: 